In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [14]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [15]:
from fastbook import *

In [16]:
from fastai.vision.all import *
path = untar_data(URLs.PASCAL_2007)

In [23]:
path

In [24]:
df = pd.read_csv(path/'train.csv')
df.head()

In [25]:
dblock = DataBlock()

In [26]:
df.iloc[:,0]

In [27]:
df.iloc[0,:]

In [28]:
dsets = dblock.datasets(df)

In [29]:
len(dsets.train),len(dsets.valid)

In [31]:
x,y = dsets.train[0]
x,y

In [32]:
x['fname']

In [33]:
x['labels']

In [34]:
dblock = DataBlock(get_x = lambda r: r['fname'], get_y = lambda r: r['labels'])
dsets = dblock.datasets(df)
dsets.train[0]

In [35]:
def get_x(r): return path/'train'/r['fname']
def get_y(r): return r['labels'].split(' ')
dblock = DataBlock(get_x = get_x, get_y = get_y)
dsets = dblock.datasets(df)
dsets.train[0]

In [36]:
dblock = DataBlock(blocks=(ImageBlock, MultiCategoryBlock),
                   get_x = get_x, get_y = get_y)
dsets = dblock.datasets(df)
dsets.train[0]

In [37]:
idxs = torch.where(dsets.train[0][1]==1.)[0]
dsets.train.vocab[idxs]

In [38]:
def splitter(df):
    train = df.index[~df['is_valid']].tolist()
    valid = df.index[df['is_valid']].tolist()
    return train,valid

dblock = DataBlock(blocks=(ImageBlock, MultiCategoryBlock),
                   splitter=splitter,
                   get_x=get_x, 
                   get_y=get_y)

dsets = dblock.datasets(df)
dsets.train[0]

In [40]:
dblock = DataBlock(blocks=(ImageBlock, MultiCategoryBlock),
                   splitter=splitter,
                   get_x=get_x, 
                   get_y=get_y,
                   item_tfms = RandomResizedCrop(128, min_scale=0.35))
dls = dblock.dataloaders(df)

In [41]:
dls.show_batch(nrows=1, ncols=3)


In [43]:
cnn_learner

In [44]:
learn = cnn_learner(dls, resnet18)

In [45]:
x,y = to_cpu(dls.train.one_batch())
activs = learn.model(x)
activs.shape

In [46]:
activs[0]

In [47]:
def binary_cross_entropy(inputs, targets):
    inputs = inputs.sigmoid()
    return -torch.where(targets==1, 1-inputs, inputs).log().mean()

In [48]:
loss_func = nn.BCEWithLogitsLoss()
loss = loss_func(activs, y)
loss

In [49]:
accuracy_multi

In [50]:
learn = cnn_learner(dls, resnet50, metrics=partial(accuracy_multi, thresh=0.2))
learn.fine_tune(3, base_lr=3e-3, freeze_epochs=4)
